## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_selection import VarianceThreshold

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

## Load labels

In [3]:
y = pd.read_csv('your_path/label.csv')
y = y['class2'].to_numpy()
print(y.shape)
print(y)

(289,)
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## Load data
Load gene.csv, methyl.csv, miRNA.csv for a cancer type, transpose the matrix and delete the first row. Row deletion because the first row contains the feature names.

In [4]:
# load .csv file gene
X_gene = pd.read_csv(r'your_path/gene.csv')
print(X_gene.shape)

# load .csv file methyl
X_methyl = pd.read_csv(r'your_path/methyl.csv')
print(X_methyl.shape)

# load .csv file miRNA
X_miRNA = pd.read_csv(r'your_path/miRNA.csv')
print(X_miRNA.shape)

(58316, 290)
(22928, 290)
(1879, 290)


In [5]:
X_gene_T = X_gene.transpose()
print(X_gene_T.shape)
print(X_gene_T)

X_methyl_T = X_methyl.transpose()
print(X_methyl_T.shape)
print(X_methyl_T)

X_miRNA_T = X_miRNA.transpose()
print(X_miRNA_T.shape)
print(X_miRNA_T)

(290, 58316)
                         0              1          2              3      \
Unnamed: 0       RP11-419K12.1  RP11-325K19.6  SNORD121B  RP11-325K19.2   
TCGA-CZ-5455-11            0.0            0.0        0.0            0.0   
TCGA-CZ-5467-11            0.0            0.0   0.172642            0.0   
TCGA-B0-5697-01            0.0            0.0        0.0            0.0   
TCGA-BP-4177-01            0.0            0.0        0.0            0.0   
...                        ...            ...        ...            ...   
TCGA-B0-5097-01            0.0            0.0   0.652605            0.0   
TCGA-CW-6088-01            0.0            0.0   0.166708            0.0   
TCGA-B2-5639-01            0.0            0.0        0.0            0.0   
TCGA-B8-A54I-01            0.0            0.0        0.0       0.013491   
TCGA-B8-A54F-01            0.0            0.0        0.0            0.0   

                         4             5         6          7      \
Unnamed: 0       

In [6]:
# GENE
# remove first row, containing names of features
X_gene_ = X_gene_T.iloc[1:, :]

# print shape and data to check if row containing names of features is removed
print('shape of X_gene: ', X_gene_.shape)
print('X_gene: \n', X_gene_.iloc[:10, :10])

# METHYL
# remove first row, containing names of features
X_methyl_ = X_methyl_T.iloc[1:, :]

# print shape and data to check if row containing names of features is removed
print('shape of X_methyl: ', X_methyl_.shape)
print('X_methyl: \n', X_methyl_.iloc[:10, :10])

# MIRNA
# remove first row, containing names of features
X_miRNA_ = X_miRNA_T.iloc[1:, :]

# print shape and data to check if row containing names of features is removed
print('shape of X_miRNA: ', X_miRNA_.shape)
print('X_miRNA: \n', X_miRNA_.iloc[:10, :10])

shape of X_gene:  (289, 58316)
X_gene: 
                         0    1         2         3    4         5         6  \
TCGA-CZ-5455-11       0.0  0.0       0.0       0.0  0.0       0.0  0.007371   
TCGA-CZ-5467-11       0.0  0.0  0.172642       0.0  0.0       0.0  0.009925   
TCGA-B0-5697-01       0.0  0.0       0.0       0.0  0.0       0.0  0.007875   
TCGA-BP-4177-01       0.0  0.0       0.0       0.0  0.0       0.0  0.004217   
TCGA-B0-4815-01  0.094263  0.0  0.397419  0.009513  0.0  0.040543  0.027416   
TCGA-B2-3924-01  0.071181  0.0       0.0       0.0  0.0  0.122461  0.062108   
TCGA-B0-4843-01       0.0  0.0  0.570368  0.013652  0.0       0.0  0.032789   
TCGA-B0-5701-01  0.037235  0.0       0.0       0.0  0.0       0.0   0.00722   
TCGA-B0-4828-01       0.0  0.0       0.0       0.0  0.0       0.0  0.003364   
TCGA-CZ-5982-01       0.0  0.0       0.0       0.0  0.0       0.0       0.0   

                        7    8         9  
TCGA-CZ-5455-11       0.0  0.0       0.0  
TCG

# Scaling

In [7]:
def scale_data(data):
    
    # compute mean of features
    mean = np.mean(data, axis=1)
       
    # compute variance of features
    variance = np.var(data, axis=1)
    
    # compute standard deviation of features
    std = np.sqrt(variance)
    
    # scale data
    scaled = (data.T - mean) / std
   
    return scaled

In [11]:
# transpose to set data to samples x features
X_gene_scaled = scale_data(X_gene_).T

X_methyl_scaled = scale_data(X_methyl_).T

X_miRNA_scaled = scale_data(X_miRNA_).T

# Feature selection VarianceThreshold

In [50]:
# set variance threshold
vt = VarianceThreshold(threshold=0.2)

# GENE
try:
    X_gene_varthr = vt.fit_transform(X_gene_scaled)
    print('Shape gene: ', X_gene_varthr.shape)
    np.savetxt("your_path/gene_FS.csv", X_gene_varthr.T, delimiter=", ")

except ValueError:
    print("No feature is strong enough to keep. Shape gene: ", X_gene_scaled.shape)
    np.savetxt("your_path/gene_FS.csv", X_gene_scaled.T, delimiter=", ")

# METHYL
try:
    X_methyl_varthr = vt.fit_transform(X_methyl_scaled)
    print('Shape methyl: ', X_methyl_varthr.shape)
    np.savetxt("your_path/methyl_FS.csv", X_methyl_varthr.T, delimiter=", ")

except ValueError:
    print("No feature is strong enough to keep. Shape methyl: ", X_methyl_scaled.shape)
    np.savetxt("your_path/methyl_FS.csv", X_methyl_scaled.T, delimiter=", ")

# MIRNA
try:
    X_miRNA_varthr = vt.fit_transform(X_miRNA_scaled)
    print('Shape miRNA: ', X_miRNA_varthr.shape)
    np.savetxt("your_path/miRNA_FS.csv", X_miRNA_varthr.T, delimiter=", ")

except ValueError:
    print("No feature is strong enough to keep. Shape miRNA: ", X_miRNA_scaled.shape)
    np.savetxt("your_path/miRNA_FS.csv", X_miRNA_scaled.T, delimiter=", ")

Shape gene:  (289, 740)
Shape methyl:  (289, 2153)
Shape miRNA:  (289, 20)


# Feature selection SelectKBest

In [116]:
# GENE
kbest = SelectKBest(mutual_info_classif, k=round(X_gene_scaled.shape[1]*0.01))
X_gene_kbest = kbest.fit_transform(X_gene_scaled, y)
print('Shape gene: ', X_gene_kbest.shape)
np.savetxt("your_path/gene_FS.csv", X_gene_kbest.T, delimiter=", ")

# METHYL
kbest = SelectKBest(mutual_info_classif, k=round(X_methyl_scaled.shape[1]*0.1))
X_methyl_kbest = kbest.fit_transform(X_methyl_scaled, y)
print('Shape methyl: ', X_methyl_kbest.shape)
np.savetxt("your_path/methyl_FS.csv", X_methyl_kbest.T, delimiter=", ")

# MIRNA
kbest = SelectKBest(mutual_info_classif, k=round(X_miRNA_scaled.shape[1]*0.01))
X_miRNA_kbest = kbest.fit_transform(X_miRNA_scaled, y)
print('Shape miRNA: ', X_miRNA_kbest.shape)
np.savetxt("your_path/miRNA_FS.csv", X_miRNA_kbest.T, delimiter=", ")

Shape gene:  (289, 583)
Shape methyl:  (289, 2293)
Shape miRNA:  (289, 19)
